In [1]:
import pandas as pd

In [2]:
import rhinoMorph

In [3]:
class PreprocessText:
    """
    텍스트 전처리 클래스
    """
    def __init__(self) -> None:
       self.rn = rhinoMorph.startRhino()

    def load_csv(self, path: str, encoding: str='cp949', header: int=0, index_col: int=0) -> pd.DataFrame:
        """
        .csv 파일을 불러오는 함수 -> pd.DataFrame
        """
        data = pd.read_csv(path, encoding=encoding, index_col=index_col, header=header)

        return data
    
    def load_excel(self, path: str) -> pd.DataFrame:
        """
        .xlsx 파일을 불러오는 함수 -> pd.DataFrame
        """
        data = pd.read_excel(path)

        return data

    def remove_mark(self, sentence: str) -> str:
        """
        특수문자 제거 함수 -> str
        """
        marks = ["?", "!", "。", "‘", "’", "“", "”", "`", "\'",
        "\"", "(", ")", "{", "}", "[", "]", "─", "『", "』", 
        ",", "ㆍ", "·", "ᆞ", ":", ";", "/", "…", "_", "~", 
        "∼", "∽", "□", "■", "▶", "◀", "◆", "▲", "◇", "◈", 
        "☎", "【", "】", "+", "-", "=", "±", "÷", "×", "*", 
        "^", ">", "<", "｜", "|", "％", "%", "&", "￦", "₩", 
        "\\", "\t", "\r\n", "\n", "＄", "$", "¥", "￥", "£", 
        "￡", "°", "㎞", "㎏", "@", "©", "ⓒ", "↑", "|", "#", 
        "♥", "♡", "★", "☆", "♪", "♬", 'ㅋ', 'ㅠ', 'ㅜ']
        sentence_no_mark = sentence
        for mark in marks:
            sentence_no_mark = sentence_no_mark.replace(mark, '')

        return sentence_no_mark
    
    def morph(self, sentence: str) -> str:
        """
        형태소 분석 함수 -> str (ex. "밥 먹다 ...")
        """
        morphed_sentence = rhinoMorph.onlyMorph_list(self.rn, sentence,
                                                     pos=['NNG', 'NNP', 'VV', 'VA', 'XR'], eomi=True)
        joined_sentence = ' '.join(morphed_sentence)

        return joined_sentence
    
    def save_to_csv(self, data: pd.DataFrame, path: str, encoding: str='cp949') -> None:
        """
        형태소 분석한 데이터 프레임 저장 함수 -> None
        """
        data.to_csv(path, encoding=encoding)
        print(f'입력받은 데이터 {len(data)} 개 저장 완료')

    def excel_to_csv(self, data_path: str, filepath: str) -> None:
        """
        excel 파일을 형태소 분석 후 csv파일로 저장
        """
        loaded_data = self.load_excel(path=data_path)
        print('불러오기 완료')
        loaded_data = self.make_one(loaded_data)
        print('빈 내용 제거 완료')
        removed_mark_series = loaded_data['리뷰'].apply(self.remove_mark)
        print('특수문자 제거 완료')
        morphed_data = removed_mark_series.apply(self.morph)
        print('형태소 분석 완료')
        save_data = pd.concat([loaded_data[['상품명', '상품 코드', '구매자 평점']], morphed_data], axis=1)
        self.save_to_csv(save_data, filepath)

    def csv_to_csv(self, data_path: str, filepath: str) -> None:
        """
        csv 파일을 형태소 분석 후 csv파일로 저장
        """
        loaded_data = self.load_csv(path=data_path)
        loaded_data = self.make_one(loaded_data)
        removed_mark_series = loaded_data['리뷰'].apply(self.remove_mark)
        morphed_data = removed_mark_series.apply(self.morph)
        save_data = pd.concat([loaded_data[['상품명', '상품 코드', '구매자 평점']], morphed_data], axis=1)
        self.save_to_csv(save_data, filepath)

    def make_one(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        [상품명, 상품번호, 별점, 리뷰제목, 리뷰내용] -> [상품명, 상품번호, 별점, 리뷰]
        """
        def remove_empty(row) -> str:
            text = ''
            if row['리뷰 제목'] != "등록된 헤드라인이 없습니다":
                text += row['리뷰 제목']

            if row['리뷰 내용'] != '등록된 리뷰내용이 없습니다':
                if text:
                    text += ' '
                text += row['리뷰 내용']
            return text
        
        first_data = data
        first_data['리뷰 내용'] = first_data['리뷰 내용'].astype(str)
        first_data['리뷰 제목'] = first_data['리뷰 제목'].astype(str)
        self.data_len = len(first_data)
        first_data['리뷰'] = first_data.apply(remove_empty, axis=1)

        adata = first_data[first_data['리뷰'] != '']
        adata.reset_index(inplace=True, drop=True)
        result = pd.concat([adata[['상품명', '상품 코드', '구매자 평점']], adata['리뷰']], axis=1)
        
        return result

In [4]:
cl = PreprocessText()

print(help(cl))

filepath:  C:\anaconda3\Lib\site-packages
classpath:  C:\anaconda3\Lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!
Help on PreprocessText in module __main__ object:

class PreprocessText(builtins.object)
 |  PreprocessText() -> None
 |  
 |  텍스트 전처리 클래스
 |  
 |  Methods defined here:
 |  
 |  __init__(self) -> None
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  csv_to_csv(self, data_path: str, filepath: str) -> None
 |      csv 파일을 형태소 분석 후 csv파일로 저장
 |  
 |  excel_to_csv(self, data_path: str, filepath: str) -> None
 |      excel 파일을 형태소 분석 후 csv파일로 저장
 |  
 |  load_csv(self, path: str, encoding: str = 'cp949', header: int = 0, index_col: int = 0) -> pandas.core.frame.DataFrame
 |      .csv 파일을 불러오는 함수 -> pd.DataFrame
 |  
 |  load_excel(self, path: str) -> pandas.core.frame.DataFrame
 |      .xlsx 파일을 불러오는 함수 -> pd.DataFrame
 |  
 |  make_one(self, data: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame
 |      [상품명, 상품번호, 별점, 리뷰제목,

In [5]:
cl.excel_to_csv(data_path='C:/Users/slugg/Documents/GitHub/temperature_of_review/쿠팡-상품리뷰-크롤링/reviwe_뷰티_스킨케어.xlsx', 
                filepath='C:/Users/slugg/Documents/GitHub/three_idiot/data/전처리_완_데이터/review_뷰티_스킨케어.csv')

불러오기 완료
빈 내용 제거 완료
특수문자 제거 완료
형태소 분석 완료
입력받은 데이터 218428 개 저장 완료


In [7]:
data = pd.read_excel('C:/Users/slugg/Documents/GitHub/temperature_of_review/쿠팡-상품리뷰-크롤링/reviwe_남성패션.xlsx')
data

: 

In [5]:
def peo(row):
    text = ''

    if row['리뷰 제목'] != "등록된 헤드라인이 없습니다":
        text += row['리뷰 제목']

    if row['리뷰 내용'] != '등록된 리뷰내용이 없습니다':
        if text:
            text += ' '
        text += row['리뷰 내용']
    
    return text

In [15]:
data = pd.read_excel('C:/Users/slugg/Documents/GitHub/temperature_of_review/쿠팡-상품리뷰-크롤링/reviwe_유아동패션.xlsx')
data

,상품명,상품 코드,구매자 평점,리뷰 제목,리뷰 내용
0,[12시전주문 오늘출발] 메르시보꾸 아동용 웰라이크상하복 바지세트 SS23W10 키...,7709810972,5,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
1,[12시전주문 오늘출발] 메르시보꾸 아동용 웰라이크상하복 바지세트 SS23W10 키...,7709810972,4,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
2,[12시전주문 오늘출발] 메르시보꾸 아동용 웰라이크상하복 바지세트 SS23W10 키...,7709810972,5,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
3,[12시전주문 오늘출발] 메르시보꾸 아동용 웰라이크상하복 바지세트 SS23W10 키...,7709810972,4,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
4,[12시전주문 오늘출발] 메르시보꾸 아동용 웰라이크상하복 바지세트 SS23W10 키...,7709810972,5,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
...,...,...,...,...,...
90589,"아동용 트위터 상하 바지세트, 머스타드, 7호",7518802603,3,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
90590,"아동용 트위터 상하 바지세트, 머스타드, 7호",7518802603,5,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
90591,"아동용 트위터 상하 바지세트, 머스타드, 7호",7518802603,5,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다
90592,"아동용 트위터 상하 바지세트, 머스타드, 5호",7518802603,5,등록된 헤드라인이 없습니다,등록된 리뷰내용이 없습니다


In [18]:
list(data['구매자 평점'].value_counts())

[70010, 13015, 4919, 1381, 1269]

In [26]:
data['리뷰 내용'] = data['리뷰 내용'].astype(str)
data['리뷰 제목'] = data['리뷰 제목'].astype(str)

data['리뷰'] = data.apply(peo, axis=1)
data['리뷰']

0         땀복같아요 키 161에 67키로 근육형이에요핏은 정말 예쁘네요집에서 잠깐 입었다 벗...
1         평소에27~28사이즈입는데 겨울이라 껴입으려고 한사이즈 업햇는데 잘맞네요 스판이잇어...
2         신축성좋고 핏이 예뻐요 평소사이즈로 주문했더니 딱맞는 핏이예요작업복으로 입으려고 주...
3         완전 이쁘고 편해요 등산복으로 구입했어요 앏은기모에 방품잘될꺼같아요신축성이 좋구 속...
4                이뻐요 평소 55입는데 허벅지 생각해서 28 주문했더니 허리도 잘 맞네요^^
                                ...                        
456153                                                     
456154                                                     
456155                                                     
456156                                                     
456157                                                     
Name: 리뷰, Length: 456158, dtype: object

In [31]:
adata = data[data['리뷰'] != '']
adata.reset_index(inplace=True, drop=True)
adata

,상품명,상품 코드,구매자 평점,리뷰 제목,리뷰 내용,리뷰
0,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,땀복같아요,키 161에 67키로 근육형이에요핏은 정말 예쁘네요집에서 잠깐 입었다 벗었는데 땀이...,땀복같아요 키 161에 67키로 근육형이에요핏은 정말 예쁘네요집에서 잠깐 입었다 벗...
1,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,등록된 헤드라인이 없습니다,평소에27~28사이즈입는데 겨울이라 껴입으려고 한사이즈 업햇는데 잘맞네요 스판이잇어...,평소에27~28사이즈입는데 겨울이라 껴입으려고 한사이즈 업햇는데 잘맞네요 스판이잇어...
2,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,신축성좋고 핏이 예뻐요,평소사이즈로 주문했더니 딱맞는 핏이예요작업복으로 입으려고 주문했는데 신축성좋고 디자...,신축성좋고 핏이 예뻐요 평소사이즈로 주문했더니 딱맞는 핏이예요작업복으로 입으려고 주...
3,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,완전 이쁘고 편해요,등산복으로 구입했어요 앏은기모에 방품잘될꺼같아요신축성이 좋구 속에 내복입고 입음 따...,완전 이쁘고 편해요 등산복으로 구입했어요 앏은기모에 방품잘될꺼같아요신축성이 좋구 속...
4,"머렐 MERRELL 여성 기모 스판 본딩 바지, 베이지, 28",7721174639,4,이뻐요,평소 55입는데 허벅지 생각해서 28 주문했더니 허리도 잘 맞네요^^,이뻐요 평소 55입는데 허벅지 생각해서 28 주문했더니 허리도 잘 맞네요^^
...,...,...,...,...,...,...
146695,"캐럿 남성용 스트레치 모크넥 니트 스웨터, L, 차콜",5938829932,5,"그레이,차콜 구매해서 입고 세탁해도 보풀도안생겨요",등록된 리뷰내용이 없습니다,"그레이,차콜 구매해서 입고 세탁해도 보풀도안생겨요"
146696,"캐럿 남성용 스트레치 모크넥 니트 스웨터, 2XL, 블랙",5938829932,4,보풀이 조금 생기는 듯 한데 깔끔해요,등록된 리뷰내용이 없습니다,보풀이 조금 생기는 듯 한데 깔끔해요
146697,"캐럿 남성용 스트레치 모크넥 니트 스웨터, 3XL, 블랙",5938829932,3,작게 나왔어요,등록된 리뷰내용이 없습니다,작게 나왔어요
146698,"캐럿 남성용 스트레치 모크넥 니트 스웨터, L, 블랙",5938829932,5,등록된 헤드라인이 없습니다,완전 깔끔하고요! 원단 완전 부드러워서 엄청 잘 입을 옷입니다. 먼저 싸이즈 : 9...,완전 깔끔하고요! 원단 완전 부드러워서 엄청 잘 입을 옷입니다. 먼저 싸이즈 : 9...


In [32]:
adata = pd.concat([adata[['상품명', '상품 코드', '구매자 평점']], adata['리뷰']], axis=1)
adata

,상품명,상품 코드,구매자 평점,리뷰
0,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,땀복같아요 키 161에 67키로 근육형이에요핏은 정말 예쁘네요집에서 잠깐 입었다 벗...
1,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,평소에27~28사이즈입는데 겨울이라 껴입으려고 한사이즈 업햇는데 잘맞네요 스판이잇어...
2,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,신축성좋고 핏이 예뻐요 평소사이즈로 주문했더니 딱맞는 핏이예요작업복으로 입으려고 주...
3,"머렐 MERRELL 여성 기모 스판 본딩 바지, 블랙, 29",7721174639,5,완전 이쁘고 편해요 등산복으로 구입했어요 앏은기모에 방품잘될꺼같아요신축성이 좋구 속...
4,"머렐 MERRELL 여성 기모 스판 본딩 바지, 베이지, 28",7721174639,4,이뻐요 평소 55입는데 허벅지 생각해서 28 주문했더니 허리도 잘 맞네요^^
...,...,...,...,...
146695,"캐럿 남성용 스트레치 모크넥 니트 스웨터, L, 차콜",5938829932,5,"그레이,차콜 구매해서 입고 세탁해도 보풀도안생겨요"
146696,"캐럿 남성용 스트레치 모크넥 니트 스웨터, 2XL, 블랙",5938829932,4,보풀이 조금 생기는 듯 한데 깔끔해요
146697,"캐럿 남성용 스트레치 모크넥 니트 스웨터, 3XL, 블랙",5938829932,3,작게 나왔어요
146698,"캐럿 남성용 스트레치 모크넥 니트 스웨터, L, 블랙",5938829932,5,완전 깔끔하고요! 원단 완전 부드러워서 엄청 잘 입을 옷입니다. 먼저 싸이즈 : 9...


In [20]:
data = pd.read_csv('C:/Users/slugg/Documents/GitHub/three_idiot/data/전처리_완_데이터/review_유아동패션.csv', encoding='cp949')

In [21]:
data

,Unnamed: 0,상품명,상품 코드,구매자 평점,리뷰
0,0,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 140",7636639833,5,화면 동일 험하다 다니다 남자아이 한철 입 생각 가격 저렴 구매 만족 세탁 후 어떻...
1,1,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 130",7636639833,5,예쁘다 두다 하다 가격 대비 남다 마른 체형 여유 잇지 사이즈 입다 내복 입다 따뜻...
2,2,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 130",7636639833,5,살다 맞다 넘다 가 따다
3,3,"월튼키즈 남아용 체커보드 플리스 상하의 세트, 블랙, 120",7636639833,1,빨다 하얗다 부분 검정 물 이염 방지 시트 넣다 빨다 입다 버리다 되다 물들다 디자...
4,4,"마리앤모리 아동용 체크 후리스 기모 상하복 세트, 120, 베이지",7714261280,5,키 몸무게 소재 가 따뜻 색상 이쁘다 서얼 굴 살다 요바 짧다 느낌 있다
...,...,...,...,...,...
23135,23135,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 90호, 옐로우",185480437,5,받다 감사
23136,23136,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 110호, 옐로우",185480437,5,배송 빠르다 옷 넘넘 귀엽다 좋다
23137,23137,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 110호, 옐로우",185480437,4,귀엽다 사이즈 크다 빠르다 배송 굳
23138,23138,"메이키즈 아동용 삐약이 7부 쟈가드 실내복, 140호, 옐로우",185480437,5,주문 여유 있다 좋다 화사 예 ㅃ 배송 하루 빠르다
